In [1]:
import pandas as pd
import os
import sys
from io import StringIO
from datetime import datetime, timedelta, time
#from scipy.integrate import trapezoid
import pvlib
import matplotlib.pyplot as plt  # for visualization

#only needed for jupyter file path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from components.SmartPowerStation import SmartPowerStation, Controls

In [2]:
CONTROLS = Controls()
CONTROLS.url = '192.168.1.116'

In [3]:
batWh = 204.8
batACW = 300

In [4]:
recentDays = await CONTROLS.getRecentData(10)

In [5]:
#merge files
def mergeDays(files):
    merged = files[-1].copy()
    for index, file in reversed(list(enumerate(files))):
        #print(index)
        if index != len(files):
            merged = pd.concat([merged,file], ignore_index=True)
    return merged

allData = mergeDays(recentDays)
allData.tail()

,datetime,powerstation_percentage,powerstation_inputWAC,powerstation_inputWDC,powerstation_outputWAC,powerstation_outputWDC,powerstation_outputMode,powerstation_deviceType,relay1_power,relay1_current,...,relay2_voltage,relay2_status,relay2_device,relay3_power,relay3_current,relay3_voltage,relay3_status,relay3_device,mode,position
6364,2025-05-11 22:52:41.242264,60.0,0.0,0.0,56.0,0.0,40.0,AC2A,0.0,0.000,...,121.5,False,Shelly2PMG3-34CDB0770E28,56.0,NaN,NaN,True,AC2A,5,NaN
6365,2025-05-11 22:55:08.244603,59.0,0.0,0.0,39.0,0.0,40.0,AC2A,0.0,0.000,...,121.4,False,Shelly2PMG3-34CDB0770E28,39.0,NaN,NaN,True,AC2A,5,NaN
6366,2025-05-11 22:58:19.232772,58.0,127.0,0.0,0.0,0.0,74.0,AC2A,29.7,0.266,...,121.2,True,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,1,NaN
6367,2025-05-11 23:00:48.245777,60.0,132.0,0.0,0.0,0.0,74.0,AC2A,30.7,0.273,...,120.4,True,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,1,NaN
6368,2025-05-11 23:03:15.220869,62.0,132.0,0.0,0.0,0.0,74.0,AC2A,98.2,0.797,...,120.4,True,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,1,NaN


## Measure Self-Consumption

Find chunks of time where DC in, AC output, and AC input are zero; measure % change over time; and average it

In [6]:
sc_DF = allData[(allData['relay3_power'] == 0) & (allData['relay2_power'] == 0) & (allData['powerstation_inputWDC'] == 0) & (allData['powerstation_outputWDC'] == 0) ][['datetime','powerstation_percentage','powerstation_inputWDC','relay2_power','relay3_power']]

In [7]:
sc_DF.head()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_power,relay3_power
1,2025-05-02 00:06:24.856228,77.0,0.0,0.0,0.0
2,2025-05-02 00:08:52.856615,77.0,0.0,0.0,0.0
3,2025-05-02 00:11:19.821429,77.0,0.0,0.0,0.0
4,2025-05-02 00:13:46.842270,77.0,0.0,0.0,0.0
5,2025-05-02 00:16:13.826729,77.0,0.0,0.0,0.0


In [8]:
sc_DF.tail()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_power,relay3_power
6342,2025-05-11 21:58:24.242911,81.0,0.0,0.0,0.0
6343,2025-05-11 22:00:51.235472,81.0,0.0,0.0,0.0
6344,2025-05-11 22:03:17.211966,81.0,0.0,0.0,0.0
6345,2025-05-11 22:05:46.250673,80.0,0.0,0.0,0.0
6346,2025-05-11 22:08:15.223346,80.0,0.0,0.0,0.0


In [9]:
sc_DF.index

Index([   1,    2,    3,    4,    5,    6,    9,   10,   11,   12,
       ...
       6336, 6337, 6339, 6340, 6341, 6342, 6343, 6344, 6345, 6346],
      dtype='int64', length=4004)

In [10]:
sc_DF.index[23:24]

Index([26], dtype='int64')

In [11]:
#to do: check if dates are contiguous!
def contiguousTimeChunks(df):
    chunks = []
    startIndex = df.index[0]
    prevIndex = startIndex -1
    temp = df.index
    for i in temp:
        #print(i)
        if prevIndex + 1 != i:   # change -1 to 25      
            chunks.append(df.loc[startIndex:prevIndex])
            startIndex = i
        if i == temp[-1]:
            print(i)
            chunks.append(df.loc[startIndex:i])
        prevIndex = i

    return chunks

sc_List = contiguousTimeChunks(sc_DF)

sc_ListF = []        
# filter out lists with only 0 values
for l in sc_List:
    pc = l['powerstation_percentage'].unique() # should have at least 2 drops in percentage
    if len(pc) > 2:
        sc_ListF.append(l)

6346


In [12]:
#convert percentage change to Wh
def WhChange(pc, bwh=batWh):
    return (bwh * (pc * .01))

In [13]:
scAn = pd.DataFrame(columns=['startDT', 'endDT','len','duration','durationS','durationH','percChange','wHChange'])

for c in sc_ListF:
    perc_change = c.iloc[0]['powerstation_percentage']- c.iloc[-1]['powerstation_percentage']
    wH_change = WhChange(perc_change, batWh) 
    dH = c.iloc[-1]['datetime'] - c.iloc[0]['datetime']
    df = pd.DataFrame({'startDT': [c.iloc[0]['datetime']], 'endDT': [c.iloc[-1]['datetime']],'len':[len(c)],'duration':[dH],'durationS':[dH.total_seconds()],'durationH':[dH.total_seconds()/60/60],'percChange':[perc_change],'wHChange':[wH_change]})
    scAn = pd.concat([scAn,df], ignore_index=True)

# scAn = pd.DataFrame(scAn).set_index('datetime')

# # Resample by 1 hour and compute the average
# hourly_avg = df.resample('1H').mean()

# print(hourly_avg)

C:\Users\alexn\AppData\Local\Temp\ipykernel_34736\1584439748.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scAn = pd.concat([scAn,df], ignore_index=True)


In [14]:
#filter out where percChange is less than 1
scAn = scAn[scAn['percChange'] >= 1]
scAn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   startDT     146 non-null    datetime64[ns] 
 1   endDT       146 non-null    datetime64[ns] 
 2   len         146 non-null    object         
 3   duration    146 non-null    timedelta64[ns]
 4   durationS   146 non-null    float64        
 5   durationH   146 non-null    float64        
 6   percChange  146 non-null    float64        
 7   wHChange    146 non-null    float64        
dtypes: datetime64[ns](2), float64(4), object(1), timedelta64[ns](1)
memory usage: 9.3+ KB


In [15]:
scAn.head()

,startDT,endDT,len,duration,durationS,durationH,percChange,wHChange
0,2025-05-02 00:26:11.853384,2025-05-02 01:29:54.826806,27,0 days 01:03:42.973422,3822.973422,1.061937,4.0,8.192
1,2025-05-02 01:37:24.857606,2025-05-02 02:38:45.856168,26,0 days 01:01:20.998562,3680.998562,1.022500,4.0,8.192
2,2025-05-02 02:46:26.836982,2025-05-02 03:47:44.824287,26,0 days 01:01:17.987305,3677.987305,1.021663,4.0,8.192
3,2025-05-02 03:55:16.848316,2025-05-02 04:54:38.845928,25,0 days 00:59:21.997612,3561.997612,0.989444,4.0,8.192
4,2025-05-02 05:02:19.862212,2025-05-02 05:31:43.827192,13,0 days 00:29:23.964980,1763.964980,0.489990,2.0,4.096


In [16]:
scAn['scW']= scAn['wHChange']/scAn['durationH']
scAn['scP']= scAn['percChange']/scAn['durationH']


In [17]:
scAn

,startDT,endDT,len,duration,durationS,durationH,percChange,wHChange,scW,scP
0,2025-05-02 00:26:11.853384,2025-05-02 01:29:54.826806,27,0 days 01:03:42.973422,3822.973422,1.061937,4.0,8.192,7.714205,3.766702
1,2025-05-02 01:37:24.857606,2025-05-02 02:38:45.856168,26,0 days 01:01:20.998562,3680.998562,1.022500,4.0,8.192,8.011739,3.911982
2,2025-05-02 02:46:26.836982,2025-05-02 03:47:44.824287,26,0 days 01:01:17.987305,3677.987305,1.021663,4.0,8.192,8.018298,3.915185
3,2025-05-02 03:55:16.848316,2025-05-02 04:54:38.845928,25,0 days 00:59:21.997612,3561.997612,0.989444,4.0,8.192,8.279399,4.042675
4,2025-05-02 05:02:19.862212,2025-05-02 05:31:43.827192,13,0 days 00:29:23.964980,1763.964980,0.489990,2.0,4.096,8.359350,4.081714
...,...,...,...,...,...,...,...,...,...,...
141,2025-05-11 07:12:31.206092,2025-05-11 08:26:24.240136,31,0 days 01:13:53.034044,4433.034044,1.231398,5.0,10.240,8.315749,4.060424
142,2025-05-11 08:33:53.223028,2025-05-11 09:30:42.205776,24,0 days 00:56:48.982748,3408.982748,0.946940,4.0,8.192,8.651026,4.224134
143,2025-05-11 19:53:19.210392,2025-05-11 20:27:48.213265,15,0 days 00:34:29.002873,2069.002873,0.574723,2.0,4.096,7.126911,3.479937
144,2025-05-11 20:32:29.208775,2025-05-11 21:04:27.250964,14,0 days 00:31:58.042189,1918.042189,0.532789,2.0,4.096,7.687839,3.753828


In [18]:
scAn.describe()

,startDT,endDT,duration,durationS,durationH,percChange,wHChange,scW,scP
count,146,146,146,146.000000,146.000000,146.000000,146.000000,146.000000,146.000000
mean,2025-05-06 11:46:29.163028992,2025-05-06 12:44:00.144355840,0 days 00:57:30.981326452,3450.981326,0.958606,4.082192,8.360329,8.873957,4.332987
min,2025-05-02 00:26:11.853384,2025-05-02 01:29:54.826806,0 days 00:14:43.983856,883.983856,0.245551,2.000000,4.096000,5.886418,2.874227
25%,2025-05-03 20:29:25.914351360,2025-05-03 21:26:17.677081856,0 days 00:37:38.503025,2258.503025,0.627362,3.000000,6.144000,7.994955,3.903787
50%,2025-05-06 04:07:48.418867456,2025-05-06 05:09:04.407129856,0 days 00:56:14.986449,3374.986449,0.937496,4.000000,8.192000,8.359350,4.081714
75%,2025-05-09 08:41:25.841102592,2025-05-09 09:05:58.599071232,0 days 01:03:40.482771500,3820.482772,1.061245,4.000000,8.192000,9.111959,4.449199
max,2025-05-11 21:10:25.214142,2025-05-11 21:44:54.248303,0 days 03:22:05.995154,12125.995154,3.368332,19.000000,38.912000,16.680848,8.144945
std,NaN,NaN,0 days 00:30:53.351821248,1853.351821,0.514820,2.385320,4.885135,1.853001,0.904786


In [19]:
# the mean scW is the average self-consumption in watts
selfConsumptionW = -scAn['scW'].mean()
print(selfConsumptionW)
selfConsumptionP = -scAn['scP'].mean()
print(selfConsumptionP)

-8.873957287691502
-4.332986956880616


# AC Out Efficiency
Find chunks where AC in is 0 and DC in is 0. (Because of the trapazoid method, we might not need to filter AC out if its in the middle of a chunk.)

Adjust for self-consumption

In [20]:
# (allData['relay3_power'] != 0) & 
acOut_DF = allData[(allData['relay2_power'] == 0) & (allData['powerstation_inputWDC'] == 0) & (allData['powerstation_outputWDC'] == 0)] #
acOut_DF = acOut_DF[['datetime','powerstation_percentage','relay1_power','relay2_power','relay3_power']]

#acOut_DF[acOut_DF['datetime'] > (datetime.now()-timedelta(days=1))]

In [21]:
acOut_DF.describe()

,datetime,powerstation_percentage,relay1_power,relay2_power,relay3_power
count,4063,4063.000000,4063.000000,4063.0,4063.000000
mean,2025-05-06 15:48:22.716965120,69.306178,9.629387,0.0,1.085405
min,2025-05-02 00:06:24.856228,11.000000,0.000000,0.0,0.000000
25%,2025-05-04 00:54:32.633442304,64.000000,1.600000,0.0,0.000000
50%,2025-05-06 17:17:00.914226944,70.000000,1.600000,0.0,0.000000
75%,2025-05-09 05:34:18.596505088,77.000000,1.700000,0.0,0.000000
max,2025-05-11 22:55:08.244603,85.000000,202.100000,0.0,214.000000
std,NaN,10.504684,21.092788,0.0,11.055357


In [22]:
acOut_DF.tail()

,datetime,powerstation_percentage,relay1_power,relay2_power,relay3_power
6361,2025-05-11 22:45:20.236715,63.0,0.0,0.0,37.0
6362,2025-05-11 22:47:48.218142,62.0,0.0,0.0,27.0
6363,2025-05-11 22:50:14.235443,61.0,0.0,0.0,27.0
6364,2025-05-11 22:52:41.242264,60.0,0.0,0.0,56.0
6365,2025-05-11 22:55:08.244603,59.0,0.0,0.0,39.0


In [23]:
# leading or trailing rows with 0s may be an issue
acO_List = contiguousTimeChunks(acOut_DF)

acO_ListF = []        
# filter out lists with only 0 values
for l in acO_List:
    m = l['relay3_power'].mean()
    #print(m)
    pc = l['powerstation_percentage'].unique() # should have at least 2 drops in percentage
    if (m > 0 ) & (len(pc) > 2):
        acO_ListF.append(l)
    

6365


In [24]:
acO_List[-1]

,datetime,powerstation_percentage,relay1_power,relay2_power,relay3_power
6339,2025-05-11 21:51:00.221481,81.0,202.1,0.0,0.0
6340,2025-05-11 21:53:29.235492,81.0,178.3,0.0,0.0
6341,2025-05-11 21:55:56.215238,81.0,181.8,0.0,0.0
6342,2025-05-11 21:58:24.242911,81.0,178.2,0.0,0.0
6343,2025-05-11 22:00:51.235472,81.0,180.9,0.0,0.0
6344,2025-05-11 22:03:17.211966,81.0,151.7,0.0,0.0
6345,2025-05-11 22:05:46.250673,80.0,118.9,0.0,0.0
6346,2025-05-11 22:08:15.223346,80.0,106.2,0.0,0.0
6347,2025-05-11 22:10:47.234518,80.0,0.0,0.0,102.0
6348,2025-05-11 22:13:15.235314,78.0,0.0,0.0,79.0


In [25]:
acoAn = pd.DataFrame(columns=['startDT', 'endDT','len','avgW','duration','durationS','durationH','percChange','WhChange','r3_Wh','sc_Wh','eff','adjEff'])

for c in acO_ListF:
    perc_change = c.iloc[-1]['powerstation_percentage'] - c.iloc[0]['powerstation_percentage']
    print(perc_change)
    wh_change =  WhChange(perc_change, batWh) # convert percent change to Wh
    print(wh_change)
    aco_Wh = float(CONTROLS.getWh(c['relay3_power'],CONTROLS.prepWh(c)['increments'])) #energy during the chunk
    print(aco_Wh)
    dur = c.iloc[-1]['datetime'] - c.iloc[0]['datetime'] #duration of chunk
    aW = c['relay3_power'].mean() #average power during chunk
    dH = dur.total_seconds()/60/60 #chunk duration in hours
    sc = selfConsumptionW * dH #self consumption during chunk (Wh)
    print(sc)
    meteredScaler = (wh_change - sc)/aco_Wh #this is what the measured energy is multiplied by to determine battery 
    e = abs(aco_Wh / (wh_change - sc)) # efficiency accounting for self-consumption
    print(e)
    ae = min(99,e*100)*.01
    print(ae)
    df = pd.DataFrame({'startDT': [c.iloc[0]['datetime']], 'endDT': [c.iloc[-1]['datetime']],'len':[len(c)],'avgW':[aW],'duration':[dur],'durationS':[dur.total_seconds()],'durationH':[dH],'percChange':[perc_change],'WhChange':[wh_change],'r3_Wh':[aco_Wh],'sc_Wh':[sc],'eff':[e],'adjEff':[ae]})
    acoAn = pd.concat([acoAn,df], ignore_index=True)
    print('')

acoAn


-9.0
-18.432
8.809901596527778
-1.0870438932186164
0.5079229686308263
0.5079229686308263

-71.0
-145.408
123.84422110430555
-11.366072797536344
0.9239215198483757
0.9239215198483757

-22.0
-45.056000000000004
37.7128107575
-9.485331340743695
1.060222148724951
0.99



C:\Users\alexn\AppData\Local\Temp\ipykernel_34736\4149052791.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  acoAn = pd.concat([acoAn,df], ignore_index=True)


,startDT,endDT,len,avgW,duration,durationS,durationH,percChange,WhChange,r3_Wh,sc_Wh,eff,adjEff
0,2025-05-05 14:10:39.553461,2025-05-05 14:18:00.547021,4,69.75000,0 days 00:07:20.993560,440.993560,0.122498,-9.0,-18.432,8.809902,-1.087044,0.507923,0.507923
1,2025-05-11 17:06:32.229477,2025-05-11 18:23:23.234550,32,93.93750,0 days 01:16:51.005073,4611.005073,1.280835,-71.0,-145.408,123.844221,-11.366073,0.923922,0.923922
2,2025-05-11 21:51:00.221481,2025-05-11 22:55:08.244603,27,34.62963,0 days 01:04:08.023122,3848.023122,1.068895,-22.0,-45.056,37.712811,-9.485331,1.060222,0.990000


In [26]:
acOutputEff = abs(acoAn['eff'].mean())
print(acOutputEff)

acOutputEffAdj = abs(acoAn['adjEff'].mean())
print(acOutputEffAdj)

0.830688879068051
0.807281496159734


# AC In Efficiency
Find chunks where AC in isn't 0, DC in is 0, and AC out is 0.

In [27]:
acIn_DF = allData[(allData['relay3_power'] == 0) & (allData['relay2_power'] != 0) & (allData['powerstation_inputWDC'] == 0) & (allData['powerstation_outputWDC'] == 0) ][['datetime','powerstation_percentage','powerstation_inputWDC','relay2_power','relay3_power']]

In [28]:
acIn_DF.head()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_power,relay3_power
0,2025-05-02 00:03:39.843487,77.0,0.0,130.0,0.0
7,2025-05-02 00:21:12.865519,76.0,0.0,129.6,0.0
8,2025-05-02 00:23:39.832091,78.0,0.0,130.9,0.0
36,2025-05-02 01:32:26.851084,76.0,0.0,130.5,0.0
37,2025-05-02 01:34:53.827661,78.0,0.0,130.9,0.0


In [29]:
acIn_DF.describe()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_power,relay3_power
count,396,396.000000,396.0,380.000000,396.0
mean,2025-05-06 20:27:21.621610752,65.515152,0.0,130.169737,0.0
min,2025-05-02 00:03:39.843487,10.000000,0.0,124.600000,0.0
25%,2025-05-03 23:37:32.392437760,61.000000,0.0,129.500000,0.0
50%,2025-05-06 10:03:38.932711936,66.000000,0.0,130.300000,0.0
75%,2025-05-09 16:46:55.852257792,76.000000,0.0,131.000000,0.0
max,2025-05-11 23:03:15.220869,83.000000,0.0,132.200000,0.0
std,NaN,13.756739,0.0,1.026923,0.0


In [30]:
# leading or trailing rows with 0s may be an issue
acI_List = contiguousTimeChunks(acIn_DF)

acI_ListF = []        
# filter out lists with only 0 values
for l in acI_List:
    m = l['relay2_power'].mean()
    pc = l['powerstation_percentage'].unique() # should have at least 2 drops in percentage
    if (m > 0 ) & (len(pc) > 2):
        acI_ListF.append(l)

6368


In [31]:
aciAn = pd.DataFrame(columns=['startDT', 'endDT','len','duration','durationS','durationH','percChange','WhChange','r2_Wh','sc_Wh','eff'])

for c in acI_ListF:
    perc_change = c.iloc[-1]['powerstation_percentage'] - c.iloc[0]['powerstation_percentage'] #when charging, start percentage is substracted from end percentage
    wh_change =  WhChange(perc_change, batWh) 
    aci_Wh = float(CONTROLS.getWh(c['relay2_power'],CONTROLS.prepWh(c)['increments']))
    dur = c.iloc[-1]['datetime'] - c.iloc[0]['datetime']
    dH = dur.total_seconds()/60/60
    sc = selfConsumptionW * dH
    e = (wh_change - sc)/ aci_Wh   #when charging, self-consumption is added to change
    df = pd.DataFrame({'startDT': [c.iloc[0]['datetime']], 'endDT': [c.iloc[-1]['datetime']],'len':[len(c)],'duration':[dur],'durationS':[dur.total_seconds()],'durationH':[dH],'percChange':[perc_change],'WhChange':[wh_change],'r2_Wh':[aci_Wh],'sc_Wh':[sc],'eff':[e]})
    aciAn = pd.concat([aciAn,df], ignore_index=True)

aciAn

C:\Users\alexn\AppData\Local\Temp\ipykernel_34736\3544961092.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aciAn = pd.concat([aciAn,df], ignore_index=True)


,startDT,endDT,len,duration,durationS,durationH,percChange,WhChange,r2_Wh,sc_Wh,eff
0,2025-05-02 19:10:24.848923,2025-05-02 19:32:26.824821,10,0 days 00:22:01.975898,1321.975898,0.367216,18.0,36.864,48.045445,-3.258655,0.835098
1,2025-05-02 19:10:24.848923,2025-05-02 19:32:26.824821,10,0 days 00:22:01.975898,1321.975898,0.367216,18.0,36.864,48.045445,-3.258655,0.835098
2,2025-05-05 14:20:36.543824,2025-05-05 14:27:54.526534,4,0 days 00:07:17.982710,437.982710,0.121662,6.0,12.288,15.625802,-1.079622,0.855484
3,2025-05-05 14:59:59.525476,2025-05-05 15:34:17.523778,15,0 days 00:34:17.998302,2057.998302,0.571666,30.0,61.440,74.954172,-5.072941,0.887381
4,2025-05-06 22:08:53.929428,2025-05-06 22:13:54.939252,3,0 days 00:05:01.009824,301.009824,0.083614,5.0,10.240,10.901884,-0.741986,1.007348
5,2025-05-07 03:22:30.792806,2025-05-07 03:28:07.809662,3,0 days 00:05:37.016856,337.016856,0.093616,5.0,10.240,12.209696,-0.830743,0.906717
6,2025-05-07 10:03:12.152857,2025-05-07 10:08:31.821639,3,0 days 00:05:19.668782,319.668782,0.088797,5.0,10.240,11.617329,-0.787980,0.949270
7,2025-05-08 23:31:02.465138,2025-05-09 00:38:42.111161,27,0 days 01:07:39.646023,4059.646023,1.127679,58.0,118.784,147.180563,-10.006979,0.875054
8,2025-05-11 18:26:01.246108,2025-05-11 19:50:48.205968,35,0 days 01:24:46.959860,5086.959860,1.413044,73.0,149.504,181.743707,-12.539296,0.891603
9,2025-05-11 22:58:19.232772,2025-05-11 23:03:15.220869,3,0 days 00:04:55.988097,295.988097,0.082219,4.0,8.192,10.575281,-0.729607,0.843628


In [32]:
acInputEff = aciAn['eff'].mean()
acInputEff

np.float64(0.8886681976648685)

# AC-in Rates

charge current at different %

In [33]:
allData.head()

,datetime,powerstation_percentage,powerstation_inputWAC,powerstation_inputWDC,powerstation_outputWAC,powerstation_outputWDC,powerstation_outputMode,powerstation_deviceType,relay1_power,relay1_current,...,relay2_voltage,relay2_status,relay2_device,relay3_power,relay3_current,relay3_voltage,relay3_status,relay3_device,mode,position
0,2025-05-02 00:03:39.843487,77.0,128.0,0.0,0.0,0.0,90.0,AC2A,57.4,0.451,...,124.7,True,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
1,2025-05-02 00:06:24.856228,77.0,0.0,0.0,0.0,0.0,56.0,AC2A,40.0,0.351,...,124.0,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
2,2025-05-02 00:08:52.856615,77.0,0.0,0.0,0.0,0.0,56.0,AC2A,42.3,0.373,...,125.4,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
3,2025-05-02 00:11:19.821429,77.0,0.0,0.0,0.0,0.0,56.0,AC2A,44.8,0.380,...,123.7,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B
4,2025-05-02 00:13:46.842270,77.0,0.0,0.0,0.0,0.0,56.0,AC2A,37.6,0.336,...,125.4,False,Shelly2PMG3-34CDB0770E28,0.0,NaN,NaN,True,AC2A,2,B


In [34]:
acInRate_DF = allData[(allData['relay3_power'] == 0) & (allData['relay2_power'] != 0) & (allData['powerstation_inputWDC'] == 0) & (allData['powerstation_outputWDC'] == 0) ][['datetime','powerstation_percentage','powerstation_inputWDC','relay2_current','relay2_power','relay3_power']]

In [35]:
acInRate_DF.head()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_current,relay2_power,relay3_power
0,2025-05-02 00:03:39.843487,77.0,0.0,1.001,130.0,0.0
7,2025-05-02 00:21:12.865519,76.0,0.0,1.008,129.6,0.0
8,2025-05-02 00:23:39.832091,78.0,0.0,1.006,130.9,0.0
36,2025-05-02 01:32:26.851084,76.0,0.0,1.000,130.5,0.0
37,2025-05-02 01:34:53.827661,78.0,0.0,1.021,130.9,0.0


In [ ]:
# # Scatter plot of relay2_current vs powerstation_percentage
# plt.figure(figsize=(8, 6))
# plt.scatter(df["relay2_current"], df["powerstation_percentage"], color='blue', marker='o')
# plt.xlabel("Relay2 Current (A)")
# plt.ylabel("Powerstation Percentage (%)")
# plt.title("Relay2 Current vs Powerstation Percentage")
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# PV Conversion Efficiency

DC-in to percentage to W

Find chunks where AC in is 0, DC in isn't 0, and AC out is 0.

In [33]:
dcIn_DF = allData[(allData['relay3_power'] == 0) & (allData['relay2_power'] == 0) & (allData['powerstation_inputWDC'] != 0) ][['datetime','powerstation_percentage','powerstation_inputWDC','relay2_power','relay3_power']]

In [34]:
dcIn_DF.describe()

,datetime,powerstation_percentage,powerstation_inputWDC,relay2_power,relay3_power
count,1608,1608.000000,1608.000000,1608.0,1608.0
mean,2025-05-06 03:13:38.463112192,74.098881,6.297886,0.0,0.0
min,2025-05-02 07:30:57.841009,31.000000,1.000000,0.0,0.0
25%,2025-05-02 16:59:17.097794816,68.000000,1.000000,0.0,0.0
50%,2025-05-05 08:49:36.534497024,76.000000,2.000000,0.0,0.0
75%,2025-05-08 16:37:12.937358848,79.000000,3.000000,0.0,0.0
max,2025-05-11 16:48:39.225056,91.000000,42.000000,0.0,0.0
std,NaN,7.960359,10.298782,0.0,0.0


In [35]:
# chunk and filter

# leading or trailing rows with 0s may be an issue
dcI_List = contiguousTimeChunks(dcIn_DF)

dcI_ListF = []        
# filter out lists with only 0 values
for l in dcI_List:
    m = l['powerstation_inputWDC'].mean()
    pc = l['powerstation_percentage'].unique() # should have at least 2 drops in percentage
    if (m > 1 ) & (len(pc) > 2):
        dcI_ListF.append(l)

#dcI_ListF

6218


In [36]:
dciAn = pd.DataFrame(columns=['startDT', 'endDT','len','duration','durationS','durationH','percChange','WhChange','dc_Wh','sc_Wh','eff'])

for c in dcI_ListF:
    perc_change = c.iloc[-1]['powerstation_percentage'] - c.iloc[0]['powerstation_percentage'] #when charging, start percentage is substracted from end percentage
    # if perc_change < 0: #drop if dc in was less than sc?
    #     continue
    wh_change =  WhChange(perc_change, batWh) 
    dci_Wh = float(CONTROLS.getWh(c['powerstation_inputWDC'],CONTROLS.prepWh(c)['increments']))

    #filter out PV inputs below 5Wh
    if dci_Wh < 5:
        continue
    dur = c.iloc[-1]['datetime'] - c.iloc[0]['datetime']
    dH = dur.total_seconds()/60/60
    sc = selfConsumptionW * dH
    e = (wh_change - sc)/ dci_Wh #when charging, self-consumption is added to change
    df = pd.DataFrame({'startDT': [c.iloc[0]['datetime']], 'endDT': [c.iloc[-1]['datetime']],'len':[len(c)],'duration':[dur],'durationS':[dur.total_seconds()],'durationH':[dH],'percChange':[perc_change],'WhChange':[wh_change],'dc_Wh':[dci_Wh],'sc_Wh':[sc],'eff':[e]})
    dciAn = pd.concat([dciAn,df], ignore_index=True)

dciAn

C:\Users\alexn\AppData\Local\Temp\ipykernel_13460\3420419629.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dciAn = pd.concat([dciAn,df], ignore_index=True)


,startDT,endDT,len,duration,durationS,durationH,percChange,WhChange,dc_Wh,sc_Wh,eff
0,2025-05-02 10:33:01.844254,2025-05-02 14:04:31.827580,87,0 days 03:31:29.983326,12689.983326,3.524995,1.0,2.048,43.848828,-31.280658,0.760081
1,2025-05-02 10:33:01.844254,2025-05-02 14:04:31.827580,87,0 days 03:31:29.983326,12689.983326,3.524995,1.0,2.048,43.848828,-31.280658,0.760081
2,2025-05-03 10:41:28.413573,2025-05-03 11:55:01.425946,31,0 days 01:13:33.012373,4413.012373,1.225837,7.0,14.336,28.206679,-10.878023,0.893903
3,2025-05-03 12:28:12.395590,2025-05-03 12:45:23.383231,8,0 days 00:17:10.987641,1030.987641,0.286385,2.0,4.096,6.810204,-2.541372,0.974622
4,2025-05-03 13:26:11.403468,2025-05-03 15:48:30.406296,59,0 days 02:22:19.002828,8539.002828,2.371945,-9.0,-18.432,6.360010,-21.048541,0.411405
5,2025-05-07 10:11:01.784079,2025-05-07 11:14:52.797060,27,0 days 01:03:51.012981,3831.012981,1.064170,-2.0,-4.096,5.378017,-9.443402,0.994307
6,2025-05-07 11:20:27.819170,2025-05-07 12:58:52.807202,41,0 days 01:38:24.988032,5904.988032,1.640274,14.0,28.672,48.377859,-14.555725,0.893544
7,2025-05-08 12:07:45.468275,2025-05-08 14:40:31.426255,63,0 days 02:32:45.957980,9165.957980,2.546099,-2.0,-4.096,20.571314,-22.593978,0.899212
8,2025-05-10 10:31:57.163468,2025-05-10 13:04:48.164880,63,0 days 02:32:51.001412,9171.001412,2.547500,14.0,28.672,53.410122,-22.606410,0.960088
9,2025-05-10 13:44:57.165065,2025-05-10 16:05:14.185754,58,0 days 02:20:17.020689,8417.020689,2.338061,-9.0,-18.432,5.903340,-20.747856,0.392296


In [37]:
dcInputEff=dciAn['eff'].mean()
dcInputEff

np.float64(0.8132386901572267)

# Inverter Model Comparison

In [38]:
#convert DC to AC with PV Watts model
#https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.inverter.pvwatts.html#pvlib.inverter.pvwatts
#args: dc power input to inverter, inverter nameplate max WAC output, nameplate efficiency

pvlib.inverter.pvwatts(70, 300,0.85)/70

np.float64(0.843856120013638)

In [39]:
pvlib.inverter.pvwatts(94, 300,0.85)/94

np.float64(0.8484072645543881)

# Validation Report

Data to be plugged in to model

In [40]:
print(f'Self-consumption: {round(selfConsumptionW,3)} W') # to do: calculate margin of error?
print(f'AC Input Efficiency: {round(acInputEff*100,3)}%')
print(f'AC Output Efficiency: {round(acOutputEff*100,3)}%')
print(f'DC Input Efficiency: {round(dcInputEff*100,3)}%')

Self-consumption: -8.874 W
AC Input Efficiency: 89.367%
AC Output Efficiency: 84.345%
DC Input Efficiency: 81.324%
